In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd

In [3]:

def build_model(input_shape=(224, 224, 3), num_classes=1):
    """Build a modified ResNet50 model with new front layers."""
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
    
    # Freeze the first half of the model
    num_layers = len(base_model.layers)
    for layer in base_model.layers[:num_layers // 2]:
        layer.trainable = True
    
    # Custom front layers
    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(num_classes, activation='softmax' if num_classes > 2 else 'sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=output_layer)
    return model

def preprocess_data(train_dir, batch_size=16, target_size=(224, 224)):
    """Preprocess images using ImageDataGenerator."""
    datagen = ImageDataGenerator(rescale=1.0/255.0, horizontal_flip=True, rotation_range=20, validation_split=0.3)
    
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='training'
    )
    
    val_generator = datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation'
    )
    
    return train_generator, val_generator

def train_and_save_model(train_dir, model_path=r'aburas_model\custom_my_model_gpt.keras', epochs=10, batch_size=16):
    """Train the model and save it."""
    train_generator, test_generator = preprocess_data(train_dir, batch_size)
    
    model = build_model()
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=epochs)
    try:
        model.save(model_path)
        print(f'Model saved to {model_path}')
    except:
        print("Model Not Saved!!")
    finally:
        return model

In [4]:
train_directory = "..//COVID Project"  # Replace with actual dataset path
model = train_and_save_model(train_directory)

Found 564 images belonging to 2 classes.


Found 140 images belonging to 2 classes.


c:\Users\HP\Documents\LLM Trainer\.llmenv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


c:\Users\HP\Documents\LLM Trainer\.llmenv\lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.4838 - loss: 15.2098

c:\Users\HP\Documents\LLM Trainer\.llmenv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 389s 10s/step - accuracy: 0.4854 - loss: 14.9736 - val_accuracy: 0.5469 - val_loss: 907.6581
Epoch 2/10
 1/35 ━━━━━━━━━━━━━━━━━━━━ 5:11 9s/step - accuracy: 0.6250 - loss: 1.0348

c:\Users\HP\Documents\LLM Trainer\.llmenv\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


35/35 ━━━━━━━━━━━━━━━━━━━━ 24s 424ms/step - accuracy: 0.6250 - loss: 1.0348 - val_accuracy: 0.4766 - val_loss: 120.0404
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 328s 10s/step - accuracy: 0.5841 - loss: 0.7223 - val_accuracy: 0.4609 - val_loss: 7.3131
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 24s 424ms/step - accuracy: 0.6875 - loss: 0.7549 - val_accuracy: 0.4609 - val_loss: 5.9917
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 326s 9s/step - accuracy: 0.6118 - loss: 0.6752 - val_accuracy: 0.4844 - val_loss: 0.6813
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 23s 421ms/step - accuracy: 0.8125 - loss: 0.4847 - val_accuracy: 0.5391 - val_loss: 0.6808
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 330s 9s/step - accuracy: 0.6843 - loss: 0.6391 - val_accuracy: 0.4453 - val_loss: 1.9491
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 25s 432ms/step - accuracy: 0.8125 - loss: 0.4288 - val_accuracy: 0.4844 - val_loss: 1.9119
Epoch 9/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 329s 9s/step - accuracy: 0.6838 - loss: 0.6401 - val_accuracy: 0.4531 - v

In [ ]:
# Replace with actual dataset path
train_and_save_model(train_directory)

In [ ]:

if __name__ == "__main__":
    train_directory = "path_to_training_data"  # Replace with actual dataset path
    train_and_save_model(train_directory)
